In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [7]:
data=pd.read_csv('Churn_Modelling.csv')

In [8]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [11]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [12]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [17]:
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))


In [18]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [19]:
data=data.drop(['Geography'],axis=1)

In [23]:
data=pd.concat([data,geo_encoded_df],axis=1)

In [24]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [27]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [28]:
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('scalerReg.pkl','wb') as f:
    pickle.dump(scaler,f)

ANN

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [35]:
Sequential_model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

/Users/harshkumar/Desktop/Main/Projects/Classification DL ANN/venv1/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
Sequential_model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [39]:
Sequential_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir="logsRegression/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [42]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [44]:
history=Sequential_model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback]
)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 654us/step - loss: 99849.0625 - mae: 99849.0625 - val_loss: 98476.3047 - val_mae: 98476.3047
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - loss: 100927.1484 - mae: 100927.1484 - val_loss: 96668.9219 - val_mae: 96668.9219
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step - loss: 96988.9375 - mae: 96988.9375 - val_loss: 92080.9688 - val_mae: 92080.9688
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step - loss: 92132.7266 - mae: 92132.7266 - val_loss: 84615.7344 - val_mae: 84615.7344
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - loss: 84186.3984 - mae: 84186.3984 - val_loss: 75319.7812 - val_mae: 75319.7812
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - loss: 74593.6172 - mae: 74593.6172 - val_loss: 66150.7344 - val_mae: 66150.7344
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - loss: 65243.0703 - mae: 65243.0703 - val_loss: 58802.8516 - val_mae: 58802.8516
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━

In [46]:
Sequential_model.save('churn_modelRegression.h5')

In [47]:
%load_ext tensorboard

In [50]:
%tensorboard --logdir logsRegression/fit/

Reusing TensorBoard on port 6020 (pid 25164), started 0:00:19 ago. (Use '!kill 25164' to kill it.)

In [52]:
## Evaluate the model
test_loss, test_mae = Sequential_model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - loss: 51087.1719 - mae: 51087.1719
Test MAE: 50306.65234375
